# 第2スクリーニング スコアリング　No. 3 & 4  
1. 企業のIRを収集してテキストマイニングを行い、DXとスーパーシティに関連する形態素を抽出する  
2. 関連語の登場回数を数え、それをスコアとする

In [1]:
#モジュールのインポート
import csv
import docx2txt
import pandas as pd
import MeCab
from wordcloud import WordCloud
import matplotlib as mpl
import matplotlib.pyplot as plt 
from pathlib import Path 
import mojimoji
import unicodedata
import urllib
import numpy as np
import seaborn as sns
import collections

In [2]:
cd /Users/shintaro/Downloads

/Users/shintaro/Downloads


In [3]:
#このモジュールは関数のコードが書かれたファイルをダウンロードして使用します
#ModuleNotFoundErrorが出たらGeneralTextMining.pyがカレントディレクトリに入っているか確認してください
import GeneralTextMining as TM

In [4]:
#ここでIRのファイル名をひとつひとつ入力していくと、画面が煩雑になってしまうのでエクセルのシート上でファイル名を入力します
comp_fstscreening = pd.read_csv("/Users/shintaro/Downloads/fst.screening2(ver1.5).csv", encoding = 'utf-8') #使用するwordファイルの名前が書かれたcsvファイルを使用します
comp_fstscreening_compname = comp_fstscreening['企業名'].tolist()
comp_fstscreening_filename = comp_fstscreening['file_name'].tolist()

In [5]:
#企業名・ファイル名がそれぞれ正確に読み込まれているか確認してください(正しく読み込まれていない場合はcsvファイルの変更内容が保存されていない可能性があるので保存してから実行してください)
comp_fstscreening

企業名                       file_name  \
0               グンゼ            2021年3月期_決算説明資料.docx   
1             日産自動車                 AR19_J_All.docx   
2                大和                             NaN   
3              ゼンリン                21smaterial.docx   
4    近鉄グループホールディングス  20210519setumeikai_siryou.docx   
..              ...                             ...   
214            ダスキン                 ダスキン(決算説明).docx   
215          タカラバイオ              タカラバイオ(統合報告書).docx   
216           三菱重工業               三菱重工業(統合報告書).docx   
217            住友化学                住友化学(統合報告書).docx   
218            北洋銀行                 北洋銀行(決算説明).docx   

                                  事業所住所  
0    /Users/shintaro/Downloads/中外鉱業.csv  
1                                   NaN  
2                                   NaN  
3                                   NaN  
4                                   NaN  
..                                  ...  
214                                 NaN  
215                                 NaN  
216                                 NaN  
217                                 NaN  
218                                 NaN  

[219 rows x 3 columns]

In [6]:
#企業の番号とファイル名が入ったリストcomp_IR_tmpを作成する
comp_IR_tmp = [[]for i in range(len(comp_fstscreening_filename))]
for i in range(len(comp_IR_tmp)):
    comp_IR_tmp[i].append(i)
    comp_IR_tmp[i].append(comp_fstscreening_filename[i])

In [7]:
#ファイルを読み込む関数read_IRを定義する
def read_IR(comp_num, file_path):
    text = docx2txt.process(file_path)
    text = text.replace('\n','').replace('\u3000','').replace('\xa0','')
    comp_IR[comp_num].append(text)

In [8]:
cd /Users/shintaro/Downloads/IR_1_44

/Users/shintaro/Downloads/IR_1_44


In [9]:
#IRのファイルがカレントディレクトリに入っていないとFileNotFoundErrorが出るので注意してください
comp_IR = [[]for i in range(len(comp_IR_tmp))]

In [10]:
#それぞれの企業のIRのテキストデータが入ったリストread_IRを作成する
for i in range(len(comp_IR_tmp)):
    if comp_IR_tmp[i][1] is np.nan:
        comp_IR[i] = []
        
    else:
        read_IR(comp_IR_tmp[i][0], comp_IR_tmp[i][1])

In [11]:
m = MeCab.Tagger()#windowsの方はこの()の中に辞書のパスを入れてください　例）('-d 辞書の場所のフルパス')

#前のstop_wordsと同様に、削除したい単語を入れてください
exwords =('もの', 'こと')

In [12]:
mining_result = [[]for i in range(len(comp_IR))]

In [13]:
#mining_resultにマイニングの結果が全て入ります
for i in range(len(comp_IR)):
    if comp_IR[i] == []:
        mining_result[i] = []
        
    else:
        res = m.parseToNode(comp_IR[i][0])
        mor = TM.morphANNew(res,combineNoun=True,useOriginal=True,psw=False)
        result = TM.POSFilter2New(mor,exwords,verbFilter=True,\
        exceptPOSs=['助詞','助動詞','動詞','連体詞','接続詞','記号','BOS/EOS'])
        c = collections.Counter(result)
        c = c.most_common(1000)
        mining_result[i] = (c)

In [14]:
#例えば、mining_result[0]とするとcsvファイルの１番目の企業のIRのマイニングの結果が表示されます
mining_result[1]

[('社外取締役', 29),
 ('1', 28),
 ('当社', 27),
 ('取締役会', 20),
 ('百万円', 16),
 ('取締役', 15),
 ('皆さま', 14),
 ('経験', 14),
 ('監督', 14),
 ('執行役', 13),
 ('監査', 13),
 ('決定', 13),
 ('年度', 12),
 ('独立性', 12),
 ('MISSIONFINANCIALHIGHLIGHTSCORPORATEGOVERNANCEMANAGEMENTPERFORMANCECONTENTSVISION',
  12),
 ('日産自動車', 11),
 ('億円', 11),
 ('経営', 11),
 ('支出', 11),
 ('ステークホルダー', 10),
 ('3', 10),
 ('構成', 10),
 ('ガバナンス', 9),
 ('執行', 9),
 ('2018年度', 8),
 ('株主', 8),
 ('JP', 8),
 ('中国合弁会社持分法ベース', 8),
 ('会社', 8),
 ('重要', 8),
 ('利益', 8),
 ('役割', 8),
 ('監査委員会', 8),
 ('見通し', 7),
 ('日産', 7),
 ('影響', 7),
 ('19', 7),
 ('自動車事業', 7),
 ('ガバナンス体制', 7),
 ('指名委員会等設置会社', 7),
 ('向上', 7),
 ('過半数', 7),
 ('委員長', 7),
 ('委員', 7),
 ('減少', 7),
 ('2', 7),
 ('前連結会計年度末', 7),
 ('将来', 6),
 ('NISSAN', 6),
 ('実現', 6),
 ('HTTPS', 6),
 ('帰属', 6),
 ('6', 6),
 ('現金同等物', 6),
 ('0004', 6),
 ('0003', 6),
 ('健全', 6),
 ('運営', 6),
 ('2019年6月', 6),
 ('移行', 6),
 ('多様', 6),
 ('視点', 6),
 ('成長', 6),
 ('監視', 6),
 ('利益相反', 6),
 ('フリーキャッシュ・フロー', 6),
 ('予想', 6),
 ('

In [15]:
#pickup_wordsに出現回数を確認したい単語を入力します(今入っている単語は本日決定したものです。他に良さそうなものがあったら提案していただけるとありがたいです。)
pickup_words_DX = ['DX','デジタルトランスフォーメーション', 'IT', 'ICT']
pickup_words_SC = ['スーパーシティ', 'スマートシティ', 'MaaS', 'SDGs', 'ICT', ]

mining_result_pickup_DX = [[]for i in range(len(mining_result))]
mining_result_pickup_SC = [[]for i in range(len(mining_result))]

In [16]:
for i in range(len(mining_result)):
    for j in range(len(mining_result[i])):
        if mining_result[i][j][0] in pickup_words_DX:
                   mining_result_pickup_DX[i].append(mining_result[i][j])
    
    #for i in range(len(mining_result[i]))

for i in range(len(mining_result)):
    for j in range(len(mining_result[i])):
        if mining_result[i][j][0] in pickup_words_SC:
                   mining_result_pickup_SC[i].append(mining_result[i][j])

In [17]:
mining_result_pickup_DX

[[],
 [],
 [],
 [],
 [('DX', 5)],
 [],
 [],
 [],
 [],
 [],
 [('IT', 4), ('DX', 3)],
 [],
 [],
 [],
 [],
 [],
 [('DX', 3)],
 [],
 [('DX', 19), ('IT', 4)],
 [],
 [('DX', 8)],
 [],
 [],
 [],
 [],
 [],
 [('DX', 6)],
 [('IT', 2)],
 [('DX', 4)],
 [('DX', 2)],
 [],
 [],
 [('DX', 31)],
 [('DX', 4)],
 [],
 [],
 [('IT', 20), ('DX', 17), ('ICT', 4), ('デジタルトランスフォーメーション', 4)],
 [('ICT', 2)],
 [],
 [],
 [],
 [('DX', 23), ('デジタルトランスフォーメーション', 5)],
 [],
 [],
 [],
 [('DX', 8), ('デジタルトランスフォーメーション', 2)],
 [('DX', 5)],
 [],
 [],
 [],
 [],
 [],
 [('DX', 10)],
 [],
 [],
 [],
 [],
 [],
 [],
 [('DX', 17), ('ICT', 13), ('デジタルトランスフォーメーション', 2)],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [('DX', 2)],
 [('IT', 8)],
 [('デジタルトランスフォーメーション', 6), ('DX', 5)],
 [],
 [('DX', 18), ('デジタルトランスフォーメーション', 4)],
 [],
 [('DX', 2)],
 [('DX', 17), ('IT', 3)],
 [('DX', 1), ('デジタルトランスフォーメーション', 1)],
 [('DX', 26), ('デジタルトランスフォーメーション', 11), ('ICT', 3)],
 [],
 [('DX', 1)],
 [],
 [],
 [('ICT', 2), ('DX', 1)],
 [('IT', 2)],
 [],

In [18]:
result_DX = [[]for i in range(len(comp_fstscreening_compname))]
result_SC = [[]for i in range(len(comp_fstscreening_compname))]

In [19]:
for i in range(len(mining_result_pickup_DX)):
    result_DX[i].append(comp_fstscreening_compname[i])
    result_DX[i].append(mining_result_pickup_DX[i])

for i in range(len(mining_result_pickup_SC)):
    result_SC[i].append(comp_fstscreening_compname[i])
    result_SC[i].append(mining_result_pickup_SC[i])

In [20]:
len_result_DX = [[]for i in range(len(mining_result_pickup_DX))]
len_result_SC = [[]for i in range(len(mining_result_pickup_SC))]

In [21]:
for i in range(len(mining_result_pickup_DX)):
    len_result_DX[i] = len(mining_result_pickup_DX[i])
    
for i in range(len(mining_result_pickup_SC)):
    len_result_SC[i] = len(mining_result_pickup_SC[i])

In [22]:
mining_result_pickup_DX[36]

[('IT', 20), ('DX', 17), ('ICT', 4), ('デジタルトランスフォーメーション', 4)]

In [23]:
max(len_result_SC)

4

In [24]:
score_DX = [[]for i in range(len(len_result_DX))]
score_SC = [[]for i in range(len(len_result_SC))]

In [25]:
for i in range(len(len_result_DX)):
    if len_result_DX[i] == 0:
        score_DX[i] = 0
        
    elif len_result_DX[i] == 1:
        score_DX[i] = mining_result_pickup_DX[i][0][1]
        
    elif len_result_DX[i] == 2:
        score_DX[i] = (mining_result_pickup_DX[i][0][1] + mining_result_pickup_DX[i][1][1])
        
    elif len_result_DX[i] == 3:
        score_DX[i] = (mining_result_pickup_DX[i][0][1] + mining_result_pickup_DX[i][1][1] + mining_result_pickup_DX[i][2][1])
        
    else:
        score_DX[i] = (mining_result_pickup_DX[i][0][1] + mining_result_pickup_DX[i][1][1] + mining_result_pickup_DX[i][2][1] + mining_result_pickup_DX[i][3][1])
    
for i in range(len(len_result_SC)):
    if len_result_SC[i] == 0:
        score_SC[i] = 0
        
    elif len_result_SC[i] == 1:
        score_SC[i] = mining_result_pickup_SC[i][0][1]
        
    elif len_result_SC[i] == 2:
        score_SC[i] = (mining_result_pickup_SC[i][0][1] + mining_result_pickup_SC[i][1][1])
        
    elif len_result_SC[i] == 3:
        score_SC[i] = (mining_result_pickup_SC[i][0][1] + mining_result_pickup_SC[i][1][1] + mining_result_pickup_SC[i][2][1])
        
    else:
        score_SC[i] = (mining_result_pickup_SC[i][0][1] + mining_result_pickup_SC[i][1][1] + mining_result_pickup_SC[i][2][1] + mining_result_pickup_SC[i][3][1])

In [26]:
score_SC

[0,
 1,
 0,
 4,
 2,
 0,
 6,
 0,
 0,
 3,
 9,
 0,
 0,
 0,
 0,
 0,
 10,
 4,
 0,
 0,
 18,
 0,
 0,
 14,
 0,
 2,
 7,
 5,
 6,
 3,
 14,
 0,
 9,
 0,
 0,
 0,
 10,
 2,
 0,
 9,
 0,
 10,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 9,
 10,
 19,
 0,
 9,
 4,
 7,
 2,
 24,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 4,
 7,
 0,
 5,
 0,
 7,
 7,
 0,
 11,
 0,
 0,
 0,
 0,
 2,
 8,
 0,
 0,
 0,
 0,
 0,
 20,
 0,
 0,
 0,
 0,
 0,
 2,
 19,
 0,
 8,
 0,
 0,
 0,
 0,
 0,
 0,
 7,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 4,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 9,
 0,
 0,
 0,
 0,
 10,
 0,
 1,
 14,
 19,
 0,
 11,
 0,
 7,
 36,
 0,
 7,
 5,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 6,
 0,
 10,
 1,
 4,
 0,
 0,
 3,
 12,
 7,
 0,
 13,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 7,
 0,
 4,
 11,
 3,
 10,
 3,
 0,
 6,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 3,
 4,
 24,
 0,
 0,
 0,
 2,
 0,
 14,
 0,
 3,
 0,
 4,
 5,
 20,
 13,
 4,
 9,
 7,
 0,
 0,
 0,
 0,
 14,
 16,
 3,
 8,
 0,
 0,
 15,
 4,
 1]

In [27]:
data_list = [[]for i in range(5)]

In [28]:
l_DX = len(mining_result_pickup_DX)
l_SC = len(mining_result_pickup_SC)

count_DX = [[]for i in range(l_DX)]
count_SC = [[]for i in range(l_SC)]

In [29]:
mining_result_pickup_DX[0]

[]

In [30]:
"""
for i in  range(l_DX):
    if mining_result_pickup_DX[i][0] != []:
        #count_DX[i] = mining_result_pickup_DX[i][1][1]
        count_DX[i] = mining_result_pickup_DX[i][0][1]
"""

'\nfor i in  range(l_DX):\n    if mining_result_pickup_DX[i][0] != []:\n        #count_DX[i] = mining_result_pickup_DX[i][1][1]\n        count_DX[i] = mining_result_pickup_DX[i][0][1]\n'

In [31]:
data_list[0] = comp_fstscreening_compname
#data_list[1] = mining_result_pickup_DX
#data_list[2] = mining_result_pickup_SC

In [32]:
column = ['企業名']

In [33]:
result_frame = pd.DataFrame(data = comp_fstscreening_compname, columns = column)

In [34]:
result_frame['DX'] = mining_result_pickup_DX

In [35]:
result_frame['SC'] = mining_result_pickup_SC

In [36]:
mining_result_pickup_SC

[[],
 [('MaaS', 1)],
 [],
 [('MaaS', 4)],
 [('MaaS', 2)],
 [],
 [('SDGs', 6)],
 [],
 [],
 [('SDGs', 3)],
 [('SDGs', 9)],
 [],
 [],
 [],
 [],
 [],
 [('SDGs', 10)],
 [('SDGs', 4)],
 [],
 [],
 [('SDGs', 18)],
 [],
 [],
 [('SDGs', 14)],
 [],
 [('SDGs', 2)],
 [('SDGs', 7)],
 [('SDGs', 5)],
 [('SDGs', 6)],
 [('MaaS', 3)],
 [('SDGs', 14)],
 [],
 [('SDGs', 9)],
 [],
 [],
 [],
 [('SDGs', 6), ('ICT', 4)],
 [('ICT', 2)],
 [],
 [('SDGs', 5), ('MaaS', 4)],
 [],
 [('SDGs', 6), ('スマートシティ', 4)],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [('MaaS', 5), ('SDGs', 4)],
 [('SDGs', 10)],
 [('SDGs', 19)],
 [],
 [('SDGs', 9)],
 [('SDGs', 4)],
 [('SDGs', 7)],
 [('SDGs', 2)],
 [('ICT', 13), ('SDGs', 11)],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [('SDGs', 2)],
 [],
 [],
 [('SDGs', 4)],
 [('SDGs', 7)],
 [],
 [('SDGs', 5)],
 [],
 [('SDGs', 7)],
 [('SDGs', 7)],
 [],
 [('SDGs', 8), ('ICT', 3)],
 [],
 [],
 [],
 [],
 [('ICT', 2)],
 [('SDGs', 5), ('スマートシティ', 3)],
 [],
 [],
 [],
 [],
 [],
 [('SDGs', 20)],
 [],
 [],


In [37]:
cd /Users/shintaro/Downloads

/Users/shintaro/Downloads


In [38]:
import wordnet_jp

word = '花'
synonym = wordnet_jp.getSynonym(word)
print(synonym)

{'flower': ['お花', 'フラワー', '花', 'ブロッサム', '御花', '華']}


In [39]:
dict = {'企業名': comp_fstscreening_compname, 'DX_Score': score_DX, 'SC_Score': score_SC}
df = pd.DataFrame(dict)

In [40]:
#df.to_csv('第二スクリーニングDX・SCスコア(ver1.3).csv')

In [41]:
df

企業名  DX_Score  SC_Score
0               グンゼ         0         0
1             日産自動車         0         1
2                大和         0         0
3              ゼンリン         0         4
4    近鉄グループホールディングス         5         2
..              ...       ...       ...
214            ダスキン         0         0
215          タカラバイオ         0         0
216           三菱重工業         2        15
217            住友化学         4         4
218            北洋銀行         0         1

[219 rows x 3 columns]

In [42]:
"""
東証上場企業名が書かれたcsvファイルをエクセルで作成し、
そのファイルを読み込んで企業名をリストwords_kigyouに格納する
"""
with open("/Users/shintaro/Downloads/kigyou.csv", encoding = "utf-8-sig") as f:
    reader = csv.reader(f)
    l = [row for row in reader]
    
words_kigyou_tmp = l[0]
words_kigyou = []

for i in range(len(words_kigyou_tmp)):
    words_kigyou.append(words_kigyou_tmp[i].replace('\u3000', ''))

In [43]:
#文を句点で分割するためのモジュールのインポート
import functools

from ja_sentence_segmenter.common.pipeline import make_pipeline
from ja_sentence_segmenter.concatenate.simple_concatenator import concatenate_matching
from ja_sentence_segmenter.normalize.neologd_normalizer import normalize
from ja_sentence_segmenter.split.simple_splitter import split_newline, split_punctuation

split_punc2 = functools.partial(split_punctuation, punctuations = r"。!?")
concat_tail_te = functools.partial(concatenate_matching, former_matching_rule = r"^(?P<result>.+)(て)$", remove_former_matched = False)
segmenter = make_pipeline(normalize, split_newline, concat_tail_te, split_punc2)

In [44]:
#分割したものをリストとして保存
lists = [[]for i in range(len(comp_IR))]

for i in range(len(lists)):
    lists[i] = list(segmenter(comp_IR[i]))

In [45]:
#リストの結合
all_lists = []
for i in range(len(lists)):
    all_lists += lists[i]

In [46]:
len(all_lists)

82939

In [47]:
#形態素解析
length = len(all_lists)
    
def get_noun_words_from_sentence(sentence, mecab, stopword_list = []):
    return [
        x.split('\t')[0] for x in mecab.parse(sentence).split('\n') if len(x.split('\t')) > 1 and \
         '名詞' in x.split('\t')[3] and x.split('\t')[0] not in stopword_list
    ]

mecab = MeCab.Tagger('-Ochasen')
nouns = [[] for i in range(length)]
stopword_list = []

for i in range(length):
    
    noun = get_noun_words_from_sentence(sentence = all_lists[i], mecab = mecab, stopword_list = stopword_list)
    nouns[i] += noun

In [48]:
len_n = len(nouns)

In [49]:
nouns_com = [[] for i in range(len_n)]

In [50]:
#分野の関連用語を入力してください
words_field = ['スーパーシティ', 'DX', 'デジタルトランスフォーメーション', 'モビリティ', '自動車', '運送', 'AI', '医療', '介護', 'ロボット', '遠隔医療']

In [51]:
#処理の軽量化のため、リストの要素のうち必要なもの以外を削除する
for i in range(len_n):
    for j in range(len(nouns[i])):
        if nouns[i][j] in words_kigyou:
            nouns_com[i].append(nouns[i][j])
            
for i in range(len_n):
    for j in range(len(nouns[i])):
        if nouns[i][j] in words_field:
            nouns_com[i].append(nouns[i][j])

In [52]:
words_wakati1 = [[]for i in range(len(comp_IR))]

In [53]:
def MeCab_textmining(num, text_tmp):
    text = text_tmp
    tagger = MeCab.Tagger("-Owakati")
    str_output = tagger.parse(text)
    words_wakati1[num] = str_output.split(' ') 

In [54]:
for i in range(len(comp_IR)):
    MeCab_textmining(i, comp_IR[i][0])

IndexError: list index out of range

In [ ]:
#words_wakati1に入っている要素の数を数えてその値をnとする
n_all = [[]for i in range(len(words_wakati1))]

for i in range(len(words_wakati1)):
    n_all[i] = len(words_wakati1[i])

In [ ]:
words_wakati_WordCloud = ''
for i in range(len(words_wakati1)):
    for j in range(len(words_wakati1[i])):
        words_wakati_WordCloud += words_wakati1[i][j]
#words_wakati_WordCloud = words_wakati_WordCloud[0]

In [ ]:
words_wakati2 = [[]for i in range(len(words_wakati1))]
words_wakati3 = [[]for i in range(len(words_wakati1))]

In [ ]:
"""
words_wakati1から要素を一つずつ順番に取り出して、その要素がwords_kigyouに含まれているのならば、
新しいリストwords_wakati2にその要素を格納するという作業をwords_wakati1の要素全てに行うことで、
words_wakati2には企業名のみが入ることになる
"""
def company_screening(num, words):
    for i in range(n_all[num]):
        if words[num][i] in words_kigyou:
            words_wakati2[num].append(words[num][i])
            
    words_wakati3[num] = ' '.join(words_wakati2[num])

In [ ]:
for j in range(len(words_wakati1)):
    company_screening(j, words_wakati1)

In [ ]:
words_wakati_WordCloud = ''
for i in range(len(words_wakati1)):
    for j in range(len(word_s_wakati1[i])):
        words_wakati_WordCloud += words_wakati1[i][j]
#words_wakati_WordCloud = words_wakati_WordCloud[0]

In [ ]:
#この部分は、先週までのWordCloudの部分と同じです
stop_words = ['コロナ', 'イノベーション', 'キャリア', 'ビジョン', 'コア', 'ベース', 'ゼロ', 'チェンジ', 'リード', '高速', 'トラスト', 'スペース'] #wordcloudの結果に入れたくないwordを決定する
fpath = '/Users/shintaro/Library/Fonts//Arial Unicode.ttf'  # 日本語フォント指定
 
wordcloud = WordCloud(
    font_path = fpath,
    width = 900, height = 600,   # default width=400, height=200
    background_color = "white",   # default=”black”
    stopwords = set(stop_words),
    max_words = 500,   # default=200
    min_font_size = 4,   #default=4
    collocations = False   #default = Ture
    ).generate(words_wakati_WordCloud)

#wordcloud.to_file("./test-dx.png") #wordcloudの結果を./以下の部分を名前としてpng形式で保存する

In [ ]:
#企業名のリストからstopwordsに該当するものを除外する
words_wakati_WordCloud1 = words_wakati_WordCloud.split()
target = stop_words
L = len(stop_words)
for i in range(L):
    
    while target[i] in words_wakati_WordCloud1:
        words_wakati_WordCloud1.remove(target[i])

In [ ]:
#同一名称の成分を除外する
wwW_unique = list(set(words_wakati_WordCloud1))
#wwW_unique

In [ ]:
#words_wakati_WordCloud2を作成
words_wakati_WordCloud2 = words_wakati_WordCloud.split()
target = stop_words
L = len(stop_words)
for i in range(L):
    
    while target[i] in words_wakati_WordCloud2:
        words_wakati_WordCloud2.remove(target[i])
        
import collections 
c = collections.Counter(words_wakati_WordCloud2)
c = c.most_common(10000)

In [ ]:
c

In [ ]:
#単語２つの組み合わせを作成する
import itertools
combination_sentences = [[] for i in range(length)]

combination_sentences = [list(itertools.combinations(words, 2)) for words in nouns_com]

In [ ]:
combination_sentences = [[tuple(sorted(combi)) for combi in combinations] for combinations in combination_sentences]

In [ ]:
tmp = []
for combinations in combination_sentences:
    tmp.extend(combinations)

In [ ]:
combination_sentences = tmp

In [ ]:
#jaccard係数を求める
def make_jaccard_coef_data(combination_sentences):

    combi_count = collections.Counter(combination_sentences)

    word_associates = []
    for key, value in combi_count.items():
        word_associates.append([key[0], key[1], value])

    word_associates = pd.DataFrame(word_associates, columns = ['word1', 'word2', 'intersection_count'])

    words = []
    for combi in combination_sentences:
        words.extend(combi)

    word_count = collections.Counter(words)
    word_count = [[key, value] for key, value in word_count.items()]
    word_count = pd.DataFrame(word_count, columns = ['word', 'count'])

    word_associates = pd.merge(
        word_associates,
        word_count.rename(columns = {'word': 'word1'}),
        on = 'word1', how = 'left'
    ).rename(columns = {'count': 'count1'}).merge(
        word_count.rename(columns = {'word': 'word2'}),
        on = 'word2', how = 'left'
    ).rename(columns = {'count': 'count2'}).assign(
        union_count = lambda x: x.count1 + x.count2 - x.intersection_count
    ).assign(jaccard_coef = lambda x: x.intersection_count / x.union_count).sort_values(
        ['jaccard_coef', 'intersection_count'], ascending = [False, False]
    )
    
    return word_associates

jaccard_coef_data = make_jaccard_coef_data(combination_sentences)

In [ ]:
#空のデータフレームjaccard_coef_data1の作成
col = ['word1',	'word2', 'intersection_count', 'count1', 'count2', 'union_count', 'jaccard_coef']
jaccard_coef_data1 = pd.DataFrame(index = [], columns = col)

In [ ]:
#二つの単語が同一の際にその行を削除する
for i in range(len(jaccard_coef_data)):
    if jaccard_coef_data.iat[i, 0] != jaccard_coef_data.iat[i, 1]:
        jaccard_coef_data1 = jaccard_coef_data1.append(jaccard_coef_data.iloc[i, :])

In [ ]:
#ジャッカード係数の下限を設定
jaccard_coef_min = 0.05

In [ ]:
#空のデータフレームjaccard_coef_data2の作成
col = ['word1',	'word2', 'intersection_count', 'count1', 'count2', 'union_count', 'jaccard_coef']
jaccard_coef_data2 = pd.DataFrame(index = [], columns = col)

In [ ]:
#ジャッカード係数の値がjaccard_coef_min以下の行を削除する
for i in range(len(jaccard_coef_data1)):
    if jaccard_coef_data1.iat[i, 6] >= jaccard_coef_min:
        jaccard_coef_data2 = jaccard_coef_data2.append(jaccard_coef_data1.iloc[i, :])

In [ ]:
#jaccard_coef_data.to_csv("jaccard_coef.csv")

In [ ]:
jaccard_coef_data2